#<font color='red'>**Atenção: Esse notebook foi criado no Google Colab**</font>

# **O algoritmo RSA**

O algoritmo RSA envolve quatro passos: geração de chaves, distribuição dessas chaves, cifragem e decifragem.

O princípio por trás do RSA são três números inteiros positivos muito grandes, chamados de "e", "d" e "n". A força da sua cifragem está na dificuldade da fatoração de "n" para obter os números "e" e "d"

# Geração das chaves
Para gerar o par de chaves, selecione dois números primos distintos, simulares em magnitude mas pouco diferente na quantidade de dígitos para tornar a fatoração difícil.

No nosso exemplo utilizaremos números pequenos para reduzir a complexidade do tutorial, uma vez que estamos interessados na lógica.

In [ ]:
p = 379
q = 487

Em seguida calcule o número "n" que é a multiplicação dos dois números primos. O número "n" será utilizado como módulo para a definição das chaves públicas e privadas

In [ ]:
n = p * q

print(n)

184573


O próximo passo é calcular a função Phi. Como "p" e "q" são primos a função Phi é definida como a multiplicação de p-1 e q-1

In [ ]:
phi_of_n = (p - 1) * (q - 1)

print(phi_of_n)

183708


Agora vamos importar algumas funções.

A primeira linha importa a função gcd que retorna o maior divisor comum entre dois números. Essa função é utilizada na geração da chave privada

A função random é utilizada para gerar um número aleatório

In [ ]:
from math import gcd
import random

Abaixo temos a definição da função que gera a chave de cifragem, utilizada para gerar a chave privada

In [ ]:
def get_encryption_key(n, phi_of_n):
    lst = [i for i in range(1, n+1)]
    e_list = []
    for i in lst:
        if (1 < i) and (i < phi_of_n):
            gcd_value = gcd(i, n)
            gcd_phi = gcd(i, phi_of_n)
            if (gcd_value == 1) and (gcd_phi == 1):
                e_list.append(i)
    if len(e_list) == 1:
        return e_list[0]
    else:
        return e_list[random.randint(1, len(e_list)-1)] 

E essa outra função gera a chave de decifragem, utilizada para gerar chave pública

In [ ]:
def get_decryption_key(e, phi_of_n):
    d_list = []
    for i in range(e * 25):
        if (e * i) % phi_of_n == 1:
            d_list.append(i)
    return d_list[random.randint(1, len(d_list) - 1)]

O código abaixo utiliza as duas funções definidas acima para gerar as chaves privada e pública

In [ ]:
e = get_encryption_key(n, phi_of_n)
d = get_decryption_key(e, phi_of_n)

# Evitar colisão
while d == e:
    d = get_decryption_key(e, phi_of_n)

chave_publica = [e, n]
chave_privada = [d, n]

print(chave_privada)
print(chave_publica)

[1095869, 184573]
[56129, 184573]


# **Distribuição das chaves**

Idealmente a chave privada deve ser gerada no próprio dispositivo onde será utilizada, como equipamentos do tipo HSM (Hardware Security Module) que não permitam sua extração. Caso seja necessária a transmissão da chave privada, deve ser utilizado um meio seguro.

Já a chave pública deve ser distribuída para que seja possível a criptografia da mensagem pela parte que envia a mensagem. Nesse tutorial não vamos precisar distribuir a chave pública pois rodaremos todo o processo nesse notebook

# **Cifragem da mensagem**

Primeiro vamos definir uma função que retorna os valores ASCII dos caracteres da mensagem

In [ ]:
import string
def text_to_digits(PT):
    pool = string.ascii_letters + string.punctuation + " "
    M = []
    for i in PT:
        M.append(pool.index(i))
    return M

E agora definimos a função que faz a cifragem da mensagem usando a chave pública

In [ ]:
def encrypt(M, chave_publica):
    return [(i ** chave_publica[0]) % chave_publica[1] for i in M]

O passo final é cifrarmos a mensagem com as funções definidas acima

In [ ]:
mensagem = "Meu teste"
mensagem_em_ascii = text_to_digits(mensagem)

mensagem_cifrada = encrypt(mensagem_em_ascii, chave_publica)
print(mensagem_cifrada)

[127198, 123289, 140933, 59084, 162653, 123289, 118952, 162653, 123289]


## **Decifragem da mensagem**

De forma semelhante a cifragem da mensagem, vamos definir uma função auxiliar que retorna os caracteres da mensagem a partir de seus valores ASCII

In [ ]:
def digits_to_text(DT):
    pool = string.ascii_letters + string.punctuation + " "
    msg = ''
    for i in DT:
        msg += pool[i]
    return msg

E agora definimos a função que faz a decifragem da mensagem

In [ ]:
def decrypt(CT, private_key):
    return [((i ** chave_privada[0]) % chave_privada[1]) for i in CT]

O passo final é utilizarmos essas funções para decifrar a mensagem

In [ ]:
mensagem_decifrage_em_ascii = decrypt(mensagem_cifrada, chave_privada) 
mensagem_aberta = original_PT = digits_to_text(mensagem_decifrage_em_ascii)

print(mensagem_aberta)

Meu teste


Dessa forma concluímos o exemplo do algoritmo RSA, definindo as chaves pública e privada, cifrando e decifrando uma mensagem com essas chaves.

# **O algoritmo de Shor**

O algoritmo de Shor é um algoritmo capaz fatorar um número, ou em outras palavras encontrar os dois números utilizados numa multiplicação para gerar um terceiro número.

Esse é justamente o passo contrário ao RSA, que a partir de dois números primos compõem o número "n" que é sua base. Como vimos na sessão anterior precisamos ter acesso a esse número "n" tanto para cifrar como para decifrar uma mensagem. Ou seja, esse número é conhecido por todas as partes.

Portanto para decifrarmos uma mensagem cifrada no RSA precisamos de acesso ao número "n", um computador quântico com capacidade de execução do Shor para esse número "n" e a mensagem cifrada.

Na sessão sobre o algoritmo RSA definidos o número "n" da nossa cifragem. A partir dele vamos obter pelo algoritmo de Shor seus fatores, que são os números base para reconstrução das chave privada e pública.

O algoritmo de Shor é explicado em diversos materiais sobre computação quântica. Na sessão referências desse notebook você entrará alguns links com a explicação.

Vamos utilizar o Cirq no Google Colab para demonstrar o uso do algoritmo de Shor e obter os números "p" e "q" que definimos no início da explicação do algoritmo RSA. 

Não entraremos na explicação de cada função definida nesse tutorial. O leitor interessado poderá consultar toda a expliacação nos links da sessão de referências

Primeiro vamos importar as funções necessárias e instalar o Cirq

In [ ]:
import fractions
import math
import random

import numpy as np
import sympy
from typing import Callable, Iterable, List, Optional, Sequence, Union

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

installing cirq...
     |████████████████████████████████| 576 kB 11.5 MB/s 
     |████████████████████████████████| 1.8 MB 50.5 MB/s 
     |████████████████████████████████| 57 kB 5.5 MB/s 
     |████████████████████████████████| 594 kB 57.1 MB/s 
     |████████████████████████████████| 66 kB 5.2 MB/s 
     |████████████████████████████████| 47 kB 5.2 MB/s 
     |████████████████████████████████| 221 kB 52.6 MB/s 
     |████████████████████████████████| 1.0 MB 66.8 MB/s 
     |████████████████████████████████| 44 kB 3.0 MB/s 
     |████████████████████████████████| 147 kB 62.3 MB/s 
     |████████████████████████████████| 229 kB 65.2 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 243 kB 49.8 MB/s 
     |████████████████████████████████| 1.6 MB 41.2 MB/s 
     |█████

Ao contrário do IBM Qiskit o Cirq ainda não possuí uma função que gera o circuito pronto para nós (ou pelo menos não localizei até agora). Portanto vamos implementar todos os passos.

A fatoração de um número inteiro pode ser obtida através da procura do período do resto da divisão de uma função exponencial pelo número que queremos fatorar.
Ao obter o período dessa função podemos calcular com alta probabilidade os fatores do número resultante da multiplicação. 

Primeiro vamos definir uma função que calcula de forma clássica o menor expoente que podemos elevar um número qualquer, desde que o resto da divisão pelo número que queremos fatorar seja 1

In [ ]:
"""Function for classically computing the order of an element of Z_n."""
def classical_order_finder(x: int, n: int) -> Optional[int]:
    """Computes smallest positive r such that x**r mod n == 1.

    Args:
        x: Integer whose order is to be computed, must be greater than one
           and belong to the multiplicative group of integers modulo n (which
           consists of positive integers relatively prime to n),
        n: Modulus of the multiplicative group.

    Returns:
        Smallest positive integer r such that x**r == 1 mod n.
        Always succeeds (and hence never returns None).

    Raises:
        ValueError when x is 1 or not an element of the multiplicative
        group of integers modulo n.
    """
    # Make sure x is both valid and in Z_n.
    if x < 2 or x >= n or math.gcd(x, n) > 1:
        raise ValueError(f"Invalid x={x} for modulus n={n}.")

    # Determine the order.
    r, y = 1, x
    while y != 1:
        y = (x * y) % n
        r += 1
    return r

Antes definir um circuito quântico que faça o mesmo cálculo, vamos definir uma função que receberá o resultado desse circuito e o interpretará através de alguns cálculos.

Esses cálculos são descritos no tutorial do algoritmo de Shor do Google Cirq apresentado na sessão de referências no final desse texto.

In [ ]:
def process_measurement(result: cirq.Result, x: int, n: int) -> Optional[int]:
    """Interprets the output of the order finding circuit.

    Specifically, it determines s/r such that exp(2πis/r) is an eigenvalue
    of the unitary

        U|y⟩ = |xy mod n⟩  0 <= y < n
        U|y⟩ = |y⟩         n <= y

    then computes r (by continued fractions) if possible, and returns it.

    Args:
        result: result obtained by sampling the output of the
            circuit built by make_order_finding_circuit

    Returns:
        r, the order of x modulo n or None.
    """
    # Read the output integer of the exponent register.
    exponent_as_integer = result.data["exponent"][0]
    exponent_num_bits = result.measurements["exponent"].shape[1]
    eigenphase = float(exponent_as_integer / 2**exponent_num_bits)

    # Run the continued fractions algorithm to determine f = s / r.
    f = fractions.Fraction.from_float(eigenphase).limit_denominator(n)

    # If the numerator is zero, the order finder failed.
    if f.numerator == 0:
        return None

    # Else, return the denominator if it is valid.
    r = f.denominator
    if x**r % n != 1:
        return None
    return r



Agora vamos definir mais duas funções.

A primeira cria o circuito quântico para fazer o cálculo do valor de um número elevado ao outro, que é a função exponencial

Já a segunda função utiliza o cálculo da primeira função e das criadas anteriormente para cálculo o resto da divisão da função exponencial pelo número que desejamos fatorar

In [ ]:
"""Function to make the quantum circuit for order finding."""
def make_order_finding_circuit(x: int, n: int) -> cirq.Circuit:
    """Returns quantum circuit which computes the order of x modulo n.

    The circuit uses Quantum Phase Estimation to compute an eigenvalue of
    the following unitary:

        U|y⟩ = |y * x mod n⟩      0 <= y < n
        U|y⟩ = |y⟩                n <= y

    Args:
        x: positive integer whose order modulo n is to be found
        n: modulus relative to which the order of x is to be found

    Returns:
        Quantum circuit for finding the order of x modulo n
    """
    L = n.bit_length()
    target = cirq.LineQubit.range(L)
    exponent = cirq.LineQubit.range(L, 3 * L + 3)

    # Create a ModularExp gate sized for these registers.
    mod_exp = ModularExp([2] * L, [2] * (2 * L + 3), x, n)

    return cirq.Circuit(
        cirq.X(target[L - 1]),
        cirq.H.on_each(*exponent),
        mod_exp.on(*target, *exponent),
        cirq.qft(*exponent, inverse=True),
        cirq.measure(*exponent, key='exponent'),
    )

def quantum_order_finder(x: int, n: int) -> Optional[int]:
    """Computes smallest positive r such that x**r mod n == 1.

    Args:
        x: integer whose order is to be computed, must be greater than one
           and belong to the multiplicative group of integers modulo n (which
           consists of positive integers relatively prime to n),
        n: modulus of the multiplicative group.
    """
    # Check that the integer x is a valid element of the multiplicative group
    # modulo n.
    if x < 2 or n <= x or math.gcd(x, n) > 1:
        raise ValueError(f'Invalid x={x} for modulus n={n}.')

    # Create the order finding circuit.
    circuit = make_order_finding_circuit(x, n)

    # Sample from the order finding circuit.
    measurement = cirq.sample(circuit)

    # Return the processed measurement result.
    return process_measurement(measurement, x, n)

O próximo passo é definimos a última função que fará uso da nossa função de fatoração quântica, adicionando algumas verificações iniciais dos parâmetros e também do resultado

In [ ]:
"""Functions for factoring from start to finish."""
def find_factor_of_prime_power(n: int) -> Optional[int]:
    """Returns non-trivial factor of n if n is a prime power, else None."""
    for k in range(2, math.floor(math.log2(n)) + 1):
        c = math.pow(n, 1 / k)
        c1 = math.floor(c)
        if c1**k == n:
            return c1
        c2 = math.ceil(c)
        if c2**k == n:
            return c2
    return None


def find_factor(
    n: int,
    order_finder: Callable[[int, int], Optional[int]] = quantum_order_finder,
    max_attempts: int = 30
) -> Optional[int]:
    """Returns a non-trivial factor of composite integer n.

    Args:
        n: Integer to factor.
        order_finder: Function for finding the order of elements of the
            multiplicative group of integers modulo n.
        max_attempts: number of random x's to try, also an upper limit
            on the number of order_finder invocations.

    Returns:
        Non-trivial factor of n or None if no such factor was found.
        Factor k of n is trivial if it is 1 or n.
    """
    # If the number is prime, there are no non-trivial factors.
    if sympy.isprime(n):
        print("n is prime!")
        return None

    # If the number is even, two is a non-trivial factor.
    if n % 2 == 0:
        return 2

    # If n is a prime power, we can find a non-trivial factor efficiently.
    c = find_factor_of_prime_power(n)
    if c is not None:
        return c

    for _ in range(max_attempts):
        # Choose a random number between 2 and n - 1.
        x = random.randint(2, n - 1)

        # Most likely x and n will be relatively prime.
        c = math.gcd(x, n)

        # If x and n are not relatively prime, we got lucky and found
        # a non-trivial factor.
        if 1 < c < n:
            return c

        # Compute the order r of x modulo n using the order finder.
        r = order_finder(x, n)

        # If the order finder failed, try again.
        if r is None:
            continue

        # If the order r is even, try again.
        if r % 2 != 0:
            continue

        # Compute the non-trivial factor.
        y = x**(r // 2) % n
        assert 1 < y < n
        c = math.gcd(y - 1, n)
        if 1 < c < n:
            return c

    print(f"Failed to find a non-trivial factor in {max_attempts} attempts.")
    return None

Por fim vamos fazer o uso das funções que definimos para obter um dos fatores do número utilizado e com isso calcular o segundo fator:

In [ ]:
# Attempt to find a factor
p = find_factor(n, order_finder=classical_order_finder)
q = n // p


print("Factoring n = pq =", n)
print("p =", p)
print("q =", q)

Factoring n = pq = 184573
p = 379
q = 487


Concluímos essa sessão com um exemplo prático da capacidade do algoritmo de Shor fatorar o número "n" do algoritmo RSA.

A partir desses fatores podemos recriar as chaves privada e pública e decifrar qualquer mensagem cifrada com a chave privada.

Porém no momento não temos um computador quântico com capacidade suficiente para fatorar o número "n" que é utilizado para proteger nossas comunicações e muitos de nossos dados em repouso.

#Criptografia pós-quântica
Porém o que ocorrerá quando tivermos um computador quântico poderoso o suficiente para quebrar o RSA e outros algoritmos de criptografia utilizados atualmente?

Para evitar esse problema estão em desenvolvimento os algoritmos de criptografia pós-quânticos. Recentemente o NIST publicou a lista dos primeiros algoritmos projetados para resistir aos computadores quânticos. Listamos alguns links sobre o assunto na sessão de referências.

Mas esse é um assunto para outro notebook.

#Referências
Algoritmo RSA: https://en.wikipedia.org/wiki/RSA_(cryptosystem)

Implementação do algoritmo RSA: https://www.codespeedy.com/rsa-algorithm-an-asymmetric-key-encryption-in-python/#:~:text=RSA%20Algorithm%20working%20example&text=Compute%20totient%20%3D%20

Algoritmo de Shor: https://en.wikipedia.org/wiki/Shor%27s_algorithm

Explicação do algoritmo de Shor: https://quantumai.google/cirq/experiments/shor

Documentação do Google Cirq: https://quantumai.google/cirq

Página inicial do Google Colab: https://colab.research.google.com/?utm_source=scs-index

Links sobre criptografia pós-quântica:

https://www.nist.gov/news-events/news/2022/07/nist-announces-first-four-quantum-resistant-cryptographic-algorithms

https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=selecionados-algoritmos-criptografia-resistentes-computadores-quanticos&id=010150220708#.YtYdgKTQ9zB

https://thequantuminsider.com/2022/07/11/crypto-quantique-announces-first-post-quantum-computing-iot-security-platform-compliant-with-new-nist-standards/